In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年3月30日～

## 積モデルの実装
TODO
* モデル式 $ T = aXYZ + b $ の実装（Model_InfiniteProductOfProblemSizeMultipliedByProcesses_ForMultipleRegression）
    * テストの実装
    * 本実装
* モデル式 $ T = a \frac{YZ}{X} + b $ の実装（Model_InfiniteProductOfProblemSizeDividedByProcesses_ForMultipleRegression）
    * テストの実装
    * 本実装

（T,X,Y,Zはそれぞれ、関数コール回数,プロセス数,問題サイズ1,問題サイズ2）

In [3]:
def test_Model_InfiniteProductOfProblemSizeMultipliedByProcesses_ForMultipleRegression():
    """test_Model_InfiniteProductOfProblemSizeMultipliedByProcesses_ForMultipleRegression()
    Model_InfiniteProductOfProblemSizeMultipliedByProcesses_ForMultipleRegressionのテスト
    """

    # 説明変数
    plotX_1 :np.ndarray = np.linspace(10, 20, 11)
    plotX_2 :np.ndarray = 10 * np.linspace(10, 20, 11)
    plotX_3 :np.ndarray = 100 * np.linspace(10, 20, 11)
    plotX_4 :np.ndarray = 1000 * np.linspace(10, 20, 11)
    plotX_5 :np.ndarray = 10000 * np.linspace(10, 20, 11)
    # 目的変数
    a :int = 10
    f :int = 50
    plotT :np.ndarray = (a * plotX_1 * plotX_2 * plotX_3 * plotX_4 * plotX_5) + f

    # DFを作成する
    # カラム名のリスト
    columnNames :list[str] = ["process", "plotX_2", "plotX_3", "plotX_4", "plotX_5", "plotT"]
    datumForDF :list[np.ndarray] = [plotX_1, plotX_2, plotX_3, plotX_4, plotX_5, plotT]
    inputDFForTest :pd.DataFrame = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 目的変数・説明変数のカラム名のリスト
    # 目的変数のカラム名のリスト
    columnNamesForExp :list[str] = ["process", "plotX_2", "plotX_3", "plotX_4", "plotX_5"]
    # 説明変数のカラム名のリスト
    columnNamesForRes :list[str] = ["plotT"]

    # 予測をする
    # モデルオブジェクトの作成
    objectModel = Model_InfiniteProductOfProblemSizeMultipliedByProcesses_ForMultipleRegression(
        inputDF=inputDFForTest,
        explanatoryVariableColumnNames=columnNamesForExp,
        responseVariableColumnNames=columnNamesForRes,
        conditionDictForTest={},
    )
    # モデルの構築
    objectModel.build_model()
    # モデル構築に用いたデータとのMAPEによって実装がうまくいっているかどうかの判定を行う
    mape = objectModel.returnMAPE()
    assert 0 <= mape < 1, f"mape = {mape}"

def test_Model_InfiniteProductOfProblemSizeDividedByProcesses_ForMultipleRegression():
    """test_Model_InfiniteProductOfProblemSizeDividedByProcesses_ForMultipleRegression()
    Model_InfiniteProductOfProblemSizeDividedByProcesses_ForMultipleRegressionのテスト
    """

    # 説明変数
    plotX_1 :np.ndarray = np.linspace(10, 20, 11)
    plotX_2 :np.ndarray = 10 * np.linspace(10, 20, 11)
    plotX_3 :np.ndarray = 100 * np.linspace(10, 20, 11)
    plotX_4 :np.ndarray = 1000 * np.linspace(10, 20, 11)
    plotX_5 :np.ndarray = 10000 * np.linspace(10, 20, 11)
    # 目的変数
    a :int = 10
    f :int = 50
    plotT :np.ndarray = a * (plotX_2 * plotX_3 * plotX_4 * plotX_5) / plotX_1  + f

    # DFを作成する
    # カラム名のリスト
    columnNames :list[str] = ["process", "plotX_2", "plotX_3", "plotX_4", "plotX_5", "plotT"]
    datumForDF :list[np.ndarray] = [plotX_1, plotX_2, plotX_3, plotX_4, plotX_5, plotT]
    inputDFForTest :pd.DataFrame = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 目的変数・説明変数のカラム名のリスト
    # 目的変数のカラム名のリスト
    columnNamesForExp :list[str] = ["process", "plotX_2", "plotX_3", "plotX_4", "plotX_5"]
    # 説明変数のカラム名のリスト
    columnNamesForRes :list[str] = ["plotT"]

    # 予測をする
    # モデルオブジェクトの作成
    objectModel = Model_InfiniteProductOfProblemSizeDividedByProcesses_ForMultipleRegression(
        inputDF=inputDFForTest,
        explanatoryVariableColumnNames=columnNamesForExp,
        responseVariableColumnNames=columnNamesForRes,
        conditionDictForTest={},
    )
    # モデルの構築
    objectModel.build_model()
    # モデル構築に用いたデータとのMAPEによって実装がうまくいっているかどうかの判定を行う
    mape = objectModel.returnMAPE()
    assert 0 <= mape < 1, f"mape = {mape}"